# Hospital Readmission Exploratory Analysis

In [ ]:
import polars as pl
import altair as alt
import kagglehub
import os
import pandas as pd



In [ ]:
# Download dataset
test_path = kagglehub.dataset_download("vanpatangan/readmission-dataset", path="test_df.csv")
train_path = kagglehub.dataset_download("vanpatangan/readmission-dataset", path="train_df.csv")
sample_submission_path = kagglehub.dataset_download("vanpatangan/readmission-dataset", path="sample_submission.csv")

print(test_path)
print(train_path)
print(sample_submission_path)


In [ ]:
test_df = pl.read_csv(test_path)
print(test_df.head())

In [ ]:
train_df = pl.read_csv(train_path)
print(train_df.head())

# Basic Descriptions of Training and Testing Data

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

# Basic Descriptions of sample submission Data

In [ ]:
sample_submission = pl.read_csv(test_path)
print(sample_submission.head())

In [ ]:
sample_submission.describe()

# Preprocessing Training Data

In [ ]:
# Preprocessing Training Data ---
#remove column readmitted from train data
train_processed = train_df.drop('readmitted')
train_processed.describe()